In [1]:
import requests as r
import getpass, pprint, time, os, cgi, json
import geopandas as gpd
import numpy as np
import shapely
from shapely import Polygon, to_geojson
from dotenv import load_dotenv

api = 'https://appeears.earthdatacloud.nasa.gov/api/'  # Set the AρρEEARS API to a variable

load_dotenv()


prods = ['MOD13A1.061', 'MOD15A2H.061', 'DAYMET.004']
layers = [(prods[0], '_500m_16_days_EVI'), (prods[0], '_500m_16_days_NDVI'), (prods[1], "Lai_500m"), (prods[2], 'prcp'), (prods[2], 'tmax'), (prods[2], 'tmin')]

task_type = ['point','area']        # Type of task, area or point
proj = 'geographic'  # Set output projection 
outFormat = ['geotiff', 'netcdf4']  # Set output file format type
recurring = False                   # Specify True for a recurring date range
#yearRange = [2000,2016]            # if recurring = True, set yearRange, change start/end date to MM-DD

params = {'limit': 1, 'pretty': True} # Limit API response to the most recent entry, return as pretty json

bounds = np.load("../data-training/ALL.npy", allow_pickle=True)
dates = np.load("../data-training/ALL-dates.npy", allow_pickle=True)

C:\Users\xubil\AppData\Local\Temp\ipykernel_15508\4106748288.py:2: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  import getpass, pprint, time, os, cgi, json


In [2]:
bounds.shape

(43433, 4)

In [3]:
curidx = np.load('../Requests/cur_idx.npy')[0]

In [4]:
curidx

21466

In [5]:
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })

for i in range(1, 5):
    user = os.getenv("EARTHDATA_USERNAME"+str(i))
    password = os.getenv("EARTHDATA_PASSWORD"+str(i))
    token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert API URL, call login service, provide credentials & return json
    token = token_response['token']                      # Save login token to a variable
    head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

    request_ids = []

    curidxbegan = curidx

    for nextidx in range(curidx, curidx+500):
        bound = bounds[nextidx]
        
        print(nextidx)

        xmin, xmax, ymin, ymax = bound[0], bound[1], bound[2], bound[3]
        # print(xmin, xmax, ymin, ymax)

        poly = Polygon([[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]])
        features = [{'type': 'Feature', 'properties': {}, 'geometry': shapely.geometry.mapping(poly)}]

        nps_gc = {"type":"FeatureCollection", "features":features}

        startDate = dates[curidx][5:] + "-" + dates[curidx][0:4]            # Start of the date range for which to extract data: MM-DD-YYYY
        endDate = startDate             # End of the date range for which to extract data: MM-DD-YYYY

        task = {
            'task_type': task_type[1],
            'task_name': "Bounds "+str(curidx),
            'params': {
                'dates': [
                {
                    'startDate': startDate,
                    'endDate': endDate
                }],
                'layers': prodLayer,
                'output': {
                        'format': {
                                'type': outFormat[0]}, 
                                'projection': proj},
                'geo': nps_gc,
            }
        }

        curidx += 1

        task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post json to the API task service, return response as json
        # task_response = {"task_id":"23132930190-4920-194-9"}
        print(task_response)

        if ('task_id' in task_response):
            task_id = task_response['task_id'] 
            request_ids.append(task_id)
        else:
            break
    
    np.save('../Requests/r'+str(i)+"_"+str(curidxbegan)+"-"+str(curidx), request_ids) # Item on right is noninclusive
    
    
np.save('../Requests/cur_idx', [curidx])



# curidx = 5466
# Saves:
# First time: 469 so 470 is next
# Second time: 470 - 966 so 967 is next
# 3: 967 - 1466
# 4: 1467 - 1966
# 5: 1967 - 2466
# 6: 2467 - 2966
# 7: 2967 - 3466
# 8: 3467 - 3751
# 9: 3752 - 3965
# 10: 3966 - 4465
# 11: 4466 - 4965
# 12: 4966 - 5465

21466
{'task_id': '5da31495-519b-4488-9b02-08a9b869f8ce', 'status': 'pending'}
21467
{'task_id': 'c5f400d7-70fe-4265-857a-cc372a49dccb', 'status': 'pending'}
21468
{'task_id': 'f9e5f068-ed8d-482d-b451-2722437c48cc', 'status': 'pending'}
21469
{'task_id': 'e66dd878-e936-42fc-b2f7-d73f50dd2b5f', 'status': 'pending'}
21470
{'task_id': '1d62cf5c-2cbe-4017-bd47-b4adfd0b08e9', 'status': 'pending'}
21471
{'task_id': 'c57f88ca-a135-417e-b497-409a1d06754f', 'status': 'pending'}
21472
{'task_id': 'db9225de-b59e-4afa-bf41-c01600f780da', 'status': 'pending'}
21473
{'task_id': 'a1ebcbf9-6314-4139-a78c-ad0371ad4fdc', 'status': 'pending'}
21474
{'task_id': '7ce06176-7eb1-4b0b-8545-32d11ed0c16b', 'status': 'pending'}
21475
{'task_id': '85495fbe-efb8-4d91-a1b0-77c774f5e9bc', 'status': 'pending'}
21476
{'task_id': '71ddae52-1521-498b-8c1b-b4f79134a866', 'status': 'pending'}
21477
{'task_id': '8947196b-f408-4971-a70f-b5bbd4171342', 'status': 'pending'}
21478
{'task_id': '354bd346-6fba-4170-8cc2-48992cfbc

In [11]:
print(task_id)

03f23543-abd7-41d5-b511-79d45ead5598


In [16]:
curidx = 12521

In [11]:
i

3

In [8]:
print(request_ids)

['a275114a-deab-4630-9d72-42a23e226e9e', '4dcccf32-6e0f-441f-808e-ecd781d8cab1', 'c0b62860-e53f-4c68-9bca-4a1fcdd7a9d0', 'da6b0f08-3fc9-4144-8b44-7cf7607e2582', '17461eb4-2b9b-43fc-83e1-379f7192cb31', '49924676-a5f9-4680-b6a5-a4639ccc558e', 'a25a9120-c43e-46d2-802d-626212c90699', 'c7013e27-2dfc-4043-833b-0d8fdd8e0e99', '9741433e-07db-4f94-8f96-6de9e37c88b1', '0a83b1d6-78b9-462a-85e1-3953763039f8', 'c82bb358-3756-4e11-b16a-194bfcfe4294', '05f95ade-fab9-493e-9818-35be8a4012bb', 'b7002b51-d8c3-4816-8060-942ce68f8991', '68992233-a27e-4b47-9361-f59062b4629a', '4c02d7d4-7a53-4851-9885-2f7b31a22f22', 'fd62e6b9-5f95-41d4-abf8-f66d03872071', '20aaa60e-a3ab-4752-8239-a72f8fce9830', '51e57cfe-c962-43ce-8ad3-90a1e55059e0', 'de0ef801-cc56-4269-90be-b4b7bdc7e46c', 'b39babc4-6260-45b6-9c25-cb468bb50f65', 'ce56143a-cc41-4834-92a2-8cca78dee2aa', 'a59408bb-e15f-497d-b520-16bc8a0f5194', '5ca2bde6-60d1-4501-99c7-02143911d595', 'a4e360e7-f846-4421-8f07-646672310d50', '79d6a71e-370f-4ed3-9a75-b5e5121040c4',

In [10]:
np.save('../Requests/r'+str(i)+"_"+str(curidxbegan)+"-"+str(curidx), request_ids)

In [7]:
len(request_ids)

444

In [9]:
curidx

16966

In [15]:
for bound in bounds:
    print(curidx) # Keep track of which indexes we need to process
    xmin, xmax, ymin, ymax = bound[0], bound[1], bound[2], bound[3]

    poly = Polygon([[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]])
    features = [{'type': 'Feature', 'properties': {}, 'geometry': shapely.geometry.mapping(poly)}]

    nps_gc = {"type":"FeatureCollection", "features":features}

    startDate = dates[curidx][5:] + "-" + dates[curidx][0:4]            # Start of the date range for which to extract data: MM-DD-YYYY
    endDate = startDate             # End of the date range for which to extract data: MM-DD-YYYY

    task = {
        'task_type': task_type[1],
        'task_name': "Bounds "+str(curidx),
        'params': {
            'dates': [
            {
                'startDate': startDate,
                'endDate': endDate
            }],
            'layers': prodLayer,
            'output': {
                    'format': {
                            'type': outFormat[0]}, 
                            'projection': proj},
            'geo': nps_gc,
        }
    }

    curidx += 1

    task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post json to the API task service, return response as json
    print(task_response)

    if ('task_id' in task_response):
        task_id = task_response['task_id'] 
        request_ids.append(task_id)
    else:
        break

    

4967
{'task_id': '18506961-98ba-4646-b5d0-a560ef89334c', 'status': 'pending'}
4968
{'task_id': '572d2a38-0112-4f2a-89c7-2163edb270a7', 'status': 'pending'}
4969
{'task_id': 'eb653b2d-0ed9-4641-ba59-10e277d95bff', 'status': 'pending'}
4970
{'task_id': '73fd8a4f-72d6-4348-b55e-73890e3e0ade', 'status': 'pending'}
4971
{'task_id': 'e5bca95e-ed65-4b8f-bf6e-0e3529045975', 'status': 'pending'}
4972
{'task_id': '8b79dbac-e994-41ac-b81a-c5267a20ce87', 'status': 'pending'}
4973
{'task_id': 'f68159dd-f06d-4c58-a762-d92a943460b4', 'status': 'pending'}
4974
{'task_id': '7cb4046b-2aea-43e5-aa9e-40469b5c4f5b', 'status': 'pending'}
4975
{'task_id': '25e3165f-ba66-4a7c-b648-54759652af48', 'status': 'pending'}
4976
{'task_id': '807061f0-be77-4013-8048-1cea7699dd06', 'status': 'pending'}
4977
{'task_id': 'd9e985b9-d2d6-4a13-9222-d9ad63134252', 'status': 'pending'}
4978
{'task_id': '50357898-1713-4da6-9603-3897652397f3', 'status': 'pending'}
4979
{'task_id': '6d8fc424-cbbe-421a-af92-58a564988288', 'status

KeyError: 'task_id'

In [1]:
len(request_ids)

NameError: name 'request_ids' is not defined

In [13]:
np.save('../Requests/request3_4966-5465', request_ids)